### Adam Sissoko
#### CS370 
#### Assignment 5 - Cartpol Problem with DQN


# Module Five Assignment: Cartpole Problem
Review the code in this notebook and in the score_logger.py file in the *scores* folder (directory). Once you have reviewed the code, return to this notebook and select **Cell** and then **Run All** from the menu bar to run this code. The code takes several minutes to run.

### Standard Run
(Below)
Solved in 134 runs, 234 total runs.

In [2]:
import random  
import gym  
import numpy as np  
from collections import deque  
from keras.models import Sequential  
from keras.layers import Dense  
from tensorflow.keras.optimizers import Adam  
  
  
from scores.score_logger import ScoreLogger  
  
ENV_NAME = "CartPole-v1"  
  
GAMMA = 0.95                # calculat the future discounted reward
LEARNING_RATE = 0.001       # how much the nn learns each iteraction
  
MEMORY_SIZE = 1000000  
BATCH_SIZE = 20             # how much memory is used to learn
  
EXPLORATION_MAX = 1.0       # rate at which agent randomly decides actions
                            # as opposed to predictions
EXPLORATION_MIN = 0.1      # explore at least this amount
EXPLORATION_DECAY = 0.995   # decrease number of explorations over time as
                            # agent ability improves
  
  
class DQNSolver:  
  
    def __init__(self, observation_space, action_space):  
        self.exploration_rate = EXPLORATION_MAX  
  
        self.action_space = action_space  
        self.memory = deque(maxlen=MEMORY_SIZE)  
  
        self.model = Sequential()  
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))  
        self.model.add(Dense(24, activation="relu"))  
        self.model.add(Dense(self.action_space, activation="linear"))  
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))  
  
    def remember(self, state, action, reward, next_state, done):  
        self.memory.append((state, action, reward, next_state, done))  
  
    def act(self, state):  
        if np.random.rand() < self.exploration_rate:  
            return random.randrange(self.action_space)  
        q_values = self.model.predict(state)  
        return np.argmax(q_values[0])  
  
    def experience_replay(self):  
        if len(self.memory) < BATCH_SIZE:  
            return  
        batch = random.sample(self.memory, BATCH_SIZE)  
        for state, action, reward, state_next, terminal in batch:  
            q_update = reward  
            if not terminal:  
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))  
            q_values = self.model.predict(state)  
            q_values[0][action] = q_update  
            self.model.fit(state, q_values, verbose=0)  
        self.exploration_rate *= EXPLORATION_DECAY  
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)  
  
  
def cartpole():  
    env = gym.make(ENV_NAME)  
    score_logger = ScoreLogger(ENV_NAME)  
    observation_space = env.observation_space.shape[0]  
    action_space = env.action_space.n  
    dqn_solver = DQNSolver(observation_space, action_space)  
    run = 0  
    while True:  
        run += 1  
        state = env.reset()  
        state = np.reshape(state, [1, observation_space])  
        step = 0  
        while True:  
            step += 1  
            #env.render()  
            action = dqn_solver.act(state)  
            state_next, reward, terminal, info = env.step(action)  
            reward = reward if not terminal else -reward  
            state_next = np.reshape(state_next, [1, observation_space])  
            dqn_solver.remember(state, action, reward, state_next, terminal)  
            state = state_next  
            if terminal:  
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))  
                score_logger.add_score(step, run)  
                break  
            dqn_solver.experience_replay()  



In [3]:
cartpole()

C:\Users\Lawrence\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\Lawrence\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


ValueError: cannot reshape array of size 2 into shape (1,4)

### Modified Run 1
In this run we changed the `LEARNING RATE = 0.001` to `0.01`and saw a significant increase in runs. A keyboard interupt was used to keep the file short, but in some tests the Run count reached over 700! In reaching for an average score of **195**, one can see how this block-run would easily reach 1000 or more Runs.  

In [ ]:
import random  
import gym  
import numpy as np  
from collections import deque  
from keras.models import Sequential  
from keras.layers import Dense  
from keras.optimizers import Adam  
  
  
from scores.score_logger import ScoreLogger  
  
ENV_NAME = "CartPole-v1"  
  
GAMMA = 0.95                # calculat the future discounted reward
LEARNING_RATE = 0.01        # how much the nn learns each iteraction
  
MEMORY_SIZE = 1000000  
BATCH_SIZE = 20             # how much memory is used to learn
  
EXPLORATION_MAX = 1.0       # rate at which agent randomly decides actions
                            # as opposed to predictions
EXPLORATION_MIN = 0.1      # explore at least this amount
EXPLORATION_DECAY = 0.995   # decrease number of explorations over time as
                            # agent ability improves
  
  
class DQNSolver:  
  
    def __init__(self, observation_space, action_space):  
        self.exploration_rate = EXPLORATION_MAX  
  
        self.action_space = action_space  
        self.memory = deque(maxlen=MEMORY_SIZE)  
  
        self.model = Sequential()  
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))  
        self.model.add(Dense(24, activation="relu"))  
        self.model.add(Dense(self.action_space, activation="linear"))  
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))  
  
    def remember(self, state, action, reward, next_state, done):  
        self.memory.append((state, action, reward, next_state, done))  
  
    def act(self, state):  
        if np.random.rand() < self.exploration_rate:  
            return random.randrange(self.action_space)  
        q_values = self.model.predict(state)  
        return np.argmax(q_values[0])  
  
    def experience_replay(self):  
        if len(self.memory) < BATCH_SIZE:  
            return  
        batch = random.sample(self.memory, BATCH_SIZE)  
        for state, action, reward, state_next, terminal in batch:  
            q_update = reward  
            if not terminal:  
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))  
            q_values = self.model.predict(state)  
            q_values[0][action] = q_update  
            self.model.fit(state, q_values, verbose=0)  
        self.exploration_rate *= EXPLORATION_DECAY  
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)  
  
  
def cartpole():  
    env = gym.make(ENV_NAME)  
    score_logger = ScoreLogger(ENV_NAME)  
    observation_space = env.observation_space.shape[0]  
    action_space = env.action_space.n  
    dqn_solver = DQNSolver(observation_space, action_space)  
    run = 0  
    while True:  
        run += 1  
        state = env.reset()  
        state = np.reshape(state, [1, observation_space])  
        step = 0  
        while True:  
            step += 1  
            #env.render()  
            action = dqn_solver.act(state)  
            state_next, reward, terminal, info = env.step(action)  
            reward = reward if not terminal else -reward  
            state_next = np.reshape(state_next, [1, observation_space])  
            dqn_solver.remember(state, action, reward, state_next, terminal)  
            state = state_next  
            if terminal:  
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))  
                score_logger.add_score(step, run)  
                break  
            dqn_solver.experience_replay()  



In [ ]:
cartpole()

### Modified Run 2
For this run we increased the `EXPLORATION_MIN = 0.1` to `0.9`. Once again the block-run was heading towards a very large total Run number (even more than the previous modification of `LEARNING_RATE`); the block was stopped at Run **302** with an average score of just **26.71**. At this rate we would be well into the 1000's of runs before reaching the goal of **195**. 

In [ ]:
import random  
import gym  
import numpy as np  
from collections import deque  
from keras.models import Sequential  
from keras.layers import Dense  
from keras.optimizers import Adam  
  
  
from scores.score_logger import ScoreLogger  
  
ENV_NAME = "CartPole-v1"  
  
GAMMA = 0.95                # calculat the future discounted reward
LEARNING_RATE = 0.001        # how much the nn learns each iteraction
  
MEMORY_SIZE = 1000000  
BATCH_SIZE = 20             # how much memory is used to learn
  
EXPLORATION_MAX = 1.0       # rate at which agent randomly decides actions
                            # as opposed to predictions
EXPLORATION_MIN = 0.9       # explore at least this amount
EXPLORATION_DECAY = 0.995   # decrease number of explorations over time as
                            # agent ability improves
  
  
class DQNSolver:  
  
    def __init__(self, observation_space, action_space):  
        self.exploration_rate = EXPLORATION_MAX  
  
        self.action_space = action_space  
        self.memory = deque(maxlen=MEMORY_SIZE)  
  
        self.model = Sequential()  
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))  
        self.model.add(Dense(24, activation="relu"))  
        self.model.add(Dense(self.action_space, activation="linear"))  
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))  
  
    def remember(self, state, action, reward, next_state, done):  
        self.memory.append((state, action, reward, next_state, done))  
  
    def act(self, state):  
        if np.random.rand() < self.exploration_rate:  
            return random.randrange(self.action_space)  
        q_values = self.model.predict(state)  
        return np.argmax(q_values[0])  
  
    def experience_replay(self):  
        if len(self.memory) < BATCH_SIZE:  
            return  
        batch = random.sample(self.memory, BATCH_SIZE)  
        for state, action, reward, state_next, terminal in batch:  
            q_update = reward  
            if not terminal:  
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))  
            q_values = self.model.predict(state)  
            q_values[0][action] = q_update  
            self.model.fit(state, q_values, verbose=0)  
        self.exploration_rate *= EXPLORATION_DECAY  
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)  
  
  
def cartpole():  
    env = gym.make(ENV_NAME)  
    score_logger = ScoreLogger(ENV_NAME)  
    observation_space = env.observation_space.shape[0]  
    action_space = env.action_space.n  
    dqn_solver = DQNSolver(observation_space, action_space)  
    run = 0  
    while True:  
        run += 1  
        state = env.reset()  
        state = np.reshape(state, [1, observation_space])  
        step = 0  
        while True:  
            step += 1  
            #env.render()  
            action = dqn_solver.act(state)  
            state_next, reward, terminal, info = env.step(action)  
            reward = reward if not terminal else -reward  
            state_next = np.reshape(state_next, [1, observation_space])  
            dqn_solver.remember(state, action, reward, state_next, terminal)  
            state = state_next  
            if terminal:  
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))  
                score_logger.add_score(step, run)  
                break  
            dqn_solver.experience_replay()  



In [ ]:
cartpole()

### Modified Run 3
For this run, we have modified the `GAMMA = 0.95` to `0.65`; the Gamma variable determines the future discounted reward for the agent. Higher values tend to push the agent towards more consideration of the total sum of the future reward when determining what actions to take during the current state. A lower value increases the agent's myopia, and a value of zero sees the agent only considering the rewards gained in the present environmental state, disregarding all future rewards from future states. 

This lower gamma value drastically increased the amount of runs (the program never even reached a 'win' state)

In [ ]:
import random  
import gym  
import numpy as np  
from collections import deque  
from keras.models import Sequential  
from keras.layers import Dense  
from keras.optimizers import Adam  
  
  
from scores.score_logger import ScoreLogger  
  
ENV_NAME = "CartPole-v1"  
  
GAMMA = 0.65                 # calculat the future discounted reward
LEARNING_RATE = 0.001        # how much the nn learns each iteraction
  
MEMORY_SIZE = 1000000  
BATCH_SIZE = 20             # how much memory is used to learn
  
EXPLORATION_MAX = 1.0       # rate at which agent randomly decides actions
                            # as opposed to predictions
EXPLORATION_MIN = 0.01       # explore at least this amount
EXPLORATION_DECAY = 0.995   # decrease number of explorations over time as
                            # agent ability improves
  
  
class DQNSolver:  
  
    def __init__(self, observation_space, action_space):  
        self.exploration_rate = EXPLORATION_MAX  
  
        self.action_space = action_space  
        self.memory = deque(maxlen=MEMORY_SIZE)  
  
        self.model = Sequential()  
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))  
        self.model.add(Dense(24, activation="relu"))  
        self.model.add(Dense(self.action_space, activation="linear"))  
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))  
  
    def remember(self, state, action, reward, next_state, done):  
        self.memory.append((state, action, reward, next_state, done))  
  
    def act(self, state):  
        if np.random.rand() < self.exploration_rate:  
            return random.randrange(self.action_space)  
        q_values = self.model.predict(state)  
        return np.argmax(q_values[0])  
  
    def experience_replay(self):  
        if len(self.memory) < BATCH_SIZE:  
            return  
        batch = random.sample(self.memory, BATCH_SIZE)  
        for state, action, reward, state_next, terminal in batch:  
            q_update = reward  
            if not terminal:  
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))  
            q_values = self.model.predict(state)  
            q_values[0][action] = q_update  
            self.model.fit(state, q_values, verbose=0)  
        self.exploration_rate *= EXPLORATION_DECAY  
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)  
  
  
def cartpole():  
    env = gym.make(ENV_NAME)  
    score_logger = ScoreLogger(ENV_NAME)  
    observation_space = env.observation_space.shape[0]  
    action_space = env.action_space.n  
    dqn_solver = DQNSolver(observation_space, action_space)  
    run = 0  
    while True:  
        run += 1  
        state = env.reset()  
        state = np.reshape(state, [1, observation_space])  
        step = 0  
        while True:  
            step += 1  
            #env.render()  
            action = dqn_solver.act(state)  
            state_next, reward, terminal, info = env.step(action)  
            reward = reward if not terminal else -reward  
            state_next = np.reshape(state_next, [1, observation_space])  
            dqn_solver.remember(state, action, reward, state_next, terminal)  
            state = state_next  
            if terminal:  
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))  
                score_logger.add_score(step, run)  
                break  
            dqn_solver.experience_replay()  



In [ ]:
cartpole()

### Analysis of the cartpole problem and code
The agent of the "cartpole" problem is tasked with balancing a pole on it's center. It does this by moving a cart left or right (the only possible actions to take) and attempting to prevent the pole from moving either direction by more than 15 degrees. The cart is expected to stay within 2.4 units of center. If either threshold is broken, the agent has failed. Each time the agent takes an action it is given feedback from the environment in form of a reward (increase in score) and a change to the next state of the environment.

To solve the cartpole problem we have employed the use of Deep Q-Learning. This learning style differs from standard Q-Learning in that a DQN uses a **Neural Network**. Where Q-Learning updates it's Q-values in the Q-table manually, the DQN uses the NN to approximate the values. With standard Q-learning the Q-value for all possible state-action pairs is manually placed in the Q-table, but with larger amounts of state-actions pairs (think thousands or even millions) this becomes infeasible. Instead, the NN is used to approximate the Q-values in the Q-table, and thus a DQN is created. The architecture of the NN is such that the states of the environment act as input while pairs of actions and Q-values are the outputs. The best possible action for the given state is represented by the output with the highest Q-value. 

In order to update the weights in the NN after each run, we use the `LEARNING_RATE` variable. Changing the LR will affect the rate of change in the weights within the model. A low learning rate typically results in more reliable training but slower optimization. Higher learning rates beget higher losses during training (see 'Modified Run 1' above). Since the goal is ultimately a decrease in loss over time, lower learning rates are preferred. 

To increase the long-term learning of the agent in the cartpole problem, Experience Replay is used within the DQN. To do this, a random sample of “experiences” is taken from the batch during each run (regulated by the `BATCH_SIZE` variable). These experiences are used to train the agent in small batches, which prevents the agent from having to train from scratch each time. This process also reduces correlation between subsequent actions, and ensures the generated Q-values are of highest quality. A discount factor (`GAMMA`) helps the agent perform better in the long-term. This is done by giving the agent a sort of “future-thinking” when considering it's actions (Wang, 2021). What this really translates to is giving the agent concern for distant future rewards as opposed to short-term rewards. An agent will consider the sum total of all future rewards forthcoming when evaluating its actions, if the gamma is equal to 1 (or very close to it); this keeps the agent “working” to always improve its own total score. A gamma of 0 will make an agent that is only concerned with actions that produce an immediate reward.


### References
* Beysolow, T. (2019). Chapter 3. In Applied Reinforcement Learning with python: With Openai Gym, tensorflow, and keras. essay, Apress.

* Gulli, A., & Pal, S. (2017). Chapter 8: AI Game Playing. In Deep learning with keras: Implement neural networks with Keras on Theano and tensorflow (pp. 271–274). essay, Packt.

* Gupta, A. (2022, May 13). Deep Q-learning. GeeksforGeeks. Retrieved September 23, 2022, from https://www.geeksforgeeks.org/deep-q-learning/

* Surma, G. (2019, November 10). Cartpole - introduction to reinforcement learning (DQN - deep Q-learning). Medium. Retrieved September 23, 2022, from https://gsurma.medium.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288

* Surmenok, P. (2021, April 19). Estimating an optimal learning rate for a deep neural network. Medium. Retrieved September 23, 2022, from https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0

* Wang, M. (2021, October 3). Deep Q-learning tutorial: Mindqn. Medium. Retrieved September 23, 2022, from https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc